# Estimating the Number of Photons Observed from a Satellite in the CO₂ 15-Micron Band

This Jupyter Notebook demonstrates the step-by-step calculations to estimate the number of photons per square meter per second observed by a satellite in the CO₂ 15-micron absorption band. We will use synthetic data to illustrate the process.

---

## **Table of Contents**

1. [Introduction](#Introduction)
2. [Constants and Parameters](#Constants-and-Parameters)
3. [Synthetic Atmospheric Data](#Synthetic-Atmospheric-Data)
4. [Calculating Molecular Populations](#Calculating-Molecular-Populations)
5. [Calculating De-excitation Probabilities](#Calculating-De-excitation-Probabilities)
6. [Calculating Absorption and Emission Coefficients](#Calculating-Absorption-and-Emission-Coefficients)
7. [Solving the Radiative Transfer Equation](#Solving-the-Radiative-Transfer-Equation)
8. [Calculating the Photon Flux](#Calculating-the-Photon-Flux)
9. [Conclusion](#Conclusion)

---

## Introduction

In this notebook, we aim to estimate the number of infrared (IR) photons emitted in the CO₂ 15-micron band that are observed by a satellite looking down through the Earth's atmosphere. We will:

- Use the Boltzmann distribution to calculate the population of CO₂ molecules in excited states.
- Determine the probabilities of radiative and collisional de-excitation.
- Calculate the absorption and emission coefficients for each atmospheric layer.
- Solve the radiative transfer equation to find the upwelling radiance.
- Convert the radiance to photon flux.

---

## Constants and Parameters

First, let's define the physical constants and parameters we'll use in our calculations.




In [1]:
# Physical Constants
h = 6.62607015e-34     # Planck's constant (J·s)
k_B = 1.380649e-23     # Boltzmann constant (J/K)
c = 2.99792458e8       # Speed of light (m/s)
N_A = 6.02214076e23    # Avogadro's number (1/mol)

# Molecular Parameters for CO2
M_CO2 = 44.01e-3       # Molar mass of CO2 (kg/mol)
m_CO2 = M_CO2 / N_A    # Mass of one CO2 molecule (kg)

# Spectral Parameters
wavelength = 15e-6     # Wavelength (15 microns) in meters
frequency = c / wavelength  # Frequency (Hz)
energy_transition = h * frequency  # Energy of the photon (J)

# Einstein Coefficient for spontaneous emission (assumed)
A21 = 1.0  # s^-1

# Collision Cross-Section (assumed)
sigma_collision = 3e-19  # m^2

# Degeneracy of the upper and lower states (assumed)
g_upper = 1
g_lower = 1

# Partition function (assumed constant for simplicity)
Z = 1
print('finished initialization')

finished initialization


In [3]:
## Synthetic Atmospheric Data

# We'll create a simplified atmosphere divided into several layers. Each layer will have synthetic values for # altitude, temperature, pressure, and CO₂ concentration.

import numpy as np
import pandas as pd

# Define atmospheric layers
layers = pd.DataFrame({
    'Layer': np.arange(1, 6),
    'Altitude': [0, 5e3, 10e3, 15e3, 20e3],  # Altitude in meters
    'Temperature': [288.15, 255.65, 223.15, 216.65, 216.65],  # Temperature in Kelvin
    'Pressure': [101325, 54019, 22632, 12036, 5474],  # Pressure in Pascals
    'CO2_Mixing_Ratio': [400e-6]*5  # CO2 mixing ratio (ppm converted to fraction)
})

# Calculate air number density in each layer (Ideal Gas Law)
layers['Number_Density'] = layers['Pressure'] / (k_B * layers['Temperature'])  # in molecules/m^3

# Calculate CO2 number density
layers['CO2_Number_Density'] = layers['Number_Density'] * layers['CO2_Mixing_Ratio']

print(layers)

   Layer  Altitude  Temperature  Pressure  CO2_Mixing_Ratio  Number_Density  \
0      1       0.0       288.15    101325            0.0004    2.546916e+25   
1      2    5000.0       255.65     54019            0.0004    1.530444e+25   
2      3   10000.0       223.15     22632            0.0004    7.345862e+24   
3      4   15000.0       216.65     12036            0.0004    4.023835e+24   
4      5   20000.0       216.65      5474            0.0004    1.830049e+24   

   CO2_Number_Density  
0        1.018767e+22  
1        6.121776e+21  
2        2.938345e+21  
3        1.609534e+21  
4        7.320198e+20  


In [10]:
## Calculating Molecular Populations

#Using the Boltzmann distribution, we'll calculate the populations of CO₂ molecules in the upper (excited) and #lower states for each layer.


# Energy levels (assuming lower state energy is zero)
E_lower = 0
E_upper = energy_transition  # Energy of the transition

# Calculate the Boltzmann factor for each layer
layers['Boltzmann_Factor'] = np.exp(-E_upper / (k_B * layers['Temperature']))

# Calculate populations (assuming degeneracy g_upper = g_lower = 1)
layers['N_upper'] = layers['CO2_Number_Density'] * layers['Boltzmann_Factor'] / Z
layers['N_lower'] = layers['CO2_Number_Density'] * (1 - layers['Boltzmann_Factor']) / Z

layers[['Layer', 'Altitude', 'Temperature', 'N_upper', 'N_lower']]


## Calculating De-excitation Probabilities

# We will calculate the probabilities of radiative and collisional de-excitation for each layer.


# Calculate the average relative speed of molecules in each layer
layers['Average_Speed'] = np.sqrt((8 * k_B * layers['Temperature']) / (np.pi * m_CO2))

# Calculate collisional rate
layers['Collisional_Rate'] = layers['Number_Density'] * sigma_collision * layers['Average_Speed']

# Radiative rate is A21 (assumed constant)
layers['Radiative_Rate'] = A21

# Total de-excitation rate
layers['Total_Deexcitation_Rate'] = layers['Collisional_Rate'] + layers['Radiative_Rate']

# Probabilities
layers['P_radiative'] = layers['Radiative_Rate'] / layers['Total_Deexcitation_Rate']
layers['P_collisional'] = layers['Collisional_Rate'] / layers['Total_Deexcitation_Rate']

layers[['Layer', 'Altitude', 'Temperature', 'P_radiative', 'P_collisional']]

## Calculating Absorption and Emission Coefficients

#Now, we'll compute the absorption coefficient (\( \kappa_\nu \)) and the effective emission coefficient (\( j_\nu^\text{eff} \)) for each layer.

# Absorption cross-section (assumed constant for simplicity)
sigma_nu = 1e-22  # m^2

# Absorption coefficient
layers['Kappa'] = (layers['N_lower'] - layers['N_upper']) * sigma_nu

# Emission coefficient (adjusted for radiative probability)
layers['Emission_Coefficient'] = layers['Kappa'] * layers['P_radiative'] * (2 * h * frequency**3) / c**2 / (np.exp(h * frequency / (k_B * layers['Temperature'])) - 1)

layers[['Layer', 'Altitude', 'Temperature', 'Kappa', 'Emission_Coefficient']]

,Layer,Altitude,Temperature,Kappa,Emission_Coefficient
0,1,0.0,288.15,0.945747,1.454572e-21
1,2,5000.0,255.65,0.583439,1.025232e-21
2,3,10000.0,223.15,0.285848,6.420543e-22
3,4,15000.0,216.65,0.157108,5.737542e-22
4,5,20000.0,216.65,0.071453,5.737542e-22


In [11]:
## Solving the Radiative Transfer Equation

#We will solve the radiative transfer equation numerically by integrating over the atmospheric layers.

# Initialize radiance
I_nu = 0

# Assume the top of the atmosphere has negligible incoming radiation at this frequency
I_nu_top = 0

# Initialize optical depth
tau_nu = 0

# Layers from top to bottom
layers_sorted = layers.sort_values(by='Altitude', ascending=False).reset_index(drop=True)

# Initialize a list to store layer contributions
layer_contributions = []

for i in range(len(layers_sorted)):
    layer = layers_sorted.loc[i]
    delta_z = (layers_sorted.loc[i-1, 'Altitude'] - layer['Altitude']) if i > 0 else (100e3 - layer['Altitude'])  # Assume atmosphere extends to 100 km
    ds = delta_z / np.cos(0)  # Zenith angle = 0 for nadir viewing
    
    # Optical depth increment
    delta_tau = layer['Kappa'] * ds
    tau_nu += delta_tau
    
    # Emission from the layer
    B_nu = (2 * h * frequency**3) / c**2 / (np.exp(h * frequency / (k_B * layer['Temperature'])) - 1)
    dI_nu = B_nu * (1 - np.exp(-delta_tau)) * np.exp(-tau_nu)
    
    # Adjust for radiative probability
    dI_nu *= layer['P_radiative']
    
    I_nu += dI_nu
    
    # Store the contribution
    layer_contributions.append({
        'Layer': layer['Layer'],
        'Altitude': layer['Altitude'],
        'Delta_Tau': delta_tau,
        'Tau_Nu': tau_nu,
        'B_Nu': B_nu,
        'dI_Nu': dI_nu,
        'I_Nu': I_nu
    })

# Convert to DataFrame for display
contributions_df = pd.DataFrame(layer_contributions)
contributions_df

,Layer,Altitude,Delta_Tau,Tau_Nu,B_Nu,dI_Nu,I_Nu
0,5.0,20000.0,5716.241061,5716.241061,1.423247e-12,0.0,0.0
1,4.0,15000.0,785.539338,6501.780400,1.423247e-12,0.0,0.0
2,3.0,10000.0,1429.238834,7931.019233,1.621851e-12,0.0,0.0
3,2.0,5000.0,2917.197408,10848.216642,2.829434e-12,0.0,0.0
4,1.0,0.0,4728.735373,15576.952015,4.375391e-12,0.0,0.0


In [12]:
## Calculating the Photon Flux

# Finally, we'll convert the upwelling radiance to photon flux.

# Upwelling radiance I_nu (from the last iteration)
I_nu_up = I_nu  # W·sr⁻¹·m⁻²·Hz⁻¹

# Convert radiance to photon flux (photons·s⁻¹·m⁻²·Hz⁻¹·sr⁻¹)
photon_flux_density = I_nu_up / (h * frequency)

# Since satellites measure over a range of frequencies and solid angles, we'll assume a small frequency interval and integrate over 2π steradians for the upward hemisphere
delta_frequency = frequency * 0.01  # 1% bandwidth
solid_angle = 2 * np.pi  # Upward hemisphere

# Total photon flux (photons·s⁻¹·m⁻²)
photon_flux = photon_flux_density * delta_frequency * solid_angle

print(f"Estimated Photon Flux: {photon_flux:.2e} photons·s⁻¹·m⁻²")


Estimated Photon Flux: 0.00e+00 photons·s⁻¹·m⁻²


## Conclusion

In this notebook, we demonstrated how to estimate the number of photons observed by a satellite in the CO₂ 15-micron band using synthetic atmospheric data. We:

- Calculated the population of CO₂ molecules in excited states using the Boltzmann distribution.
- Determined the probabilities of radiative vs. collisional de-excitation.
- Computed the absorption and emission coefficients for each atmospheric layer.
- Solved the radiative transfer equation numerically to find the upwelling radiance.
- Converted the radiance to photon flux observable by a satellite.

This simplified model provides insight into the complex processes involved in atmospheric radiative transfer and can be expanded with more detailed data and advanced techniques for more accurate estimations.

---

**Note:** The values and assumptions used in this notebook are for illustrative purposes and may not represent actual atmospheric conditions. For precise calculations, real atmospheric data and more sophisticated radiative transfer models should be used.